In [1]:
import warnings
from warnings import filterwarnings
import math
import scanpy as sc

import pandas as pd
import numpy as np
from scipy import io
import gzip
import os 
import pandas as pd
from scipy.io import mmread
import time
import re
import random
from tqdm import tqdm
import anndata
import anndata as ad
import h5py
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix
filterwarnings("ignore")
seed = 0
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# GSE197177 Pancreastic cancer 

In [3]:
sample_to_pred_label = {
    "Patient1":[0,10,11,12,13,3,6,9],
    "Patient2":[16],
    "Patient3":[13,18,8],
}


interest_pred_label  =  {
    "Patient1":[0,10,11,12,13,3,6,9],
    "Patient2":[16],
    "Patient3":[13,18,8],
}
directories = ['Patient1','Patient2','Patient3']
import pandas as pd
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix
import scanpy as sc
import os
from scipy.io import mmwrite
from tqdm import tqdm

#directories = ['Patient1','Patient2','Patient4','Patient5','Patient6','Patient7','Patient8']



from collections import Counter

def get_top_genes_attention_matrices(patient_id, gene_names, cell_names, pred_label, label, interest_label):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE197177/Patient_subsample/{directories[patient_id]}/Metastasis_Result/Attention/'
    
    
    all_genes_attention = []
    

    interest_indices = [i for i, (pred, actual) in enumerate(zip(pred_label, label)) if pred in interest_label and actual == "Metastatic"]
    print(f'\nNumber of cells of interest: {len(interest_indices)}\n')
    
    for head in range(1, 9):
        attention_matrix_path = file_path + f'head-{head}.mtx'
        attention_matrix = mmread(attention_matrix_path)
        attention_matrix_csr = csr_matrix(attention_matrix)
        attention_matrix_filtered = attention_matrix_csr[:, interest_indices].transpose()
        df_attention_filtered = pd.DataFrame(attention_matrix_filtered.toarray(), index=np.array(cell_names)[interest_indices], columns=np.array(gene_names))
        
        
        top_100_genes_attention = df_attention_filtered.abs().mean(axis=0).nlargest(100)
        

        all_genes_attention.append(top_100_genes_attention)
    
    
    df_all_genes_attention = pd.concat(all_genes_attention, axis=1)

    df_all_genes_attention.columns = [f'Head{head}' for head in range(1, 9)]
    
    return df_all_genes_attention


all_samples_genes_attention = pd.DataFrame()


for i in tqdm(range(0,len(directories))):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE197177/Patient_subsample/'+ str(directories[i]) + '/'
    ip_file = file_path + 'Metastasis_Result/'  
    
    pred_label = list(np.load(ip_file+'pred.npy'))

    interested_labels = interest_pred_label
    

    barcodes = pd.read_csv(file_path+'cell_barcodes.csv')

    features = pd.read_csv(file_path+'gene_symbol.csv')
 
    label = list(pd.read_csv(file_path+'cell_label.csv')['Pid'])
    label = ["Primary" if "P" in l else "Metastatic" if "M" in l else l for l in label]
    
    gene_names = list(features['1'])
    cell_names = list(barcodes['0'])   
    
    interest_label = sample_to_pred_label[directories[i]]
    df_all_genes_attention = get_top_genes_attention_matrices(i, gene_names, cell_names, pred_label, label, interest_label)
    

    all_samples_genes_attention = pd.concat([all_samples_genes_attention, df_all_genes_attention])

  0%|          | 0/3 [00:00<?, ?it/s]


Number of cells of interest: 5485



 33%|███▎      | 1/3 [00:07<00:15,  7.70s/it]


Number of cells of interest: 19



 67%|██████▋   | 2/3 [00:09<00:04,  4.34s/it]


Number of cells of interest: 428



100%|██████████| 3/3 [00:11<00:00,  3.93s/it]


In [8]:
all_samples_genes_attention_unique.to_csv("Pancreastic_M_N.csv")

# GSE241184 Papillary thyroid cancer 

In [6]:
interest_pred_label = [3,13,20, 26,28]

from collections import Counter

def get_top_genes_attention_matrices(patient_id, gene_names, cell_names, pred_label, label, interest_label):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE241184/Metastasis_Result/Attention/'
    
    all_genes_attention = []
    
    interest_indices = [i for i, (pred, actual) in enumerate(zip(pred_label, label)) if pred in interest_label and actual == "Metastatic"]
    print(f'\nNumber of cells of interest: {len(interest_indices)}\n')
    
    for head in range(1, 9):
        attention_matrix_path = file_path + f'head-{head}.mtx'
        attention_matrix = mmread(attention_matrix_path)
        attention_matrix_csr = csr_matrix(attention_matrix)
        attention_matrix_filtered = attention_matrix_csr[:, interest_indices].transpose()
        df_attention_filtered = pd.DataFrame(attention_matrix_filtered.toarray(), index=np.array(cell_names)[interest_indices], columns=np.array(gene_names))
        
        top_100_genes_attention = df_attention_filtered.abs().mean(axis=0).nlargest(100)
        
        all_genes_attention.append(top_100_genes_attention)
    
    df_all_genes_attention = pd.concat(all_genes_attention, axis=1)
    df_all_genes_attention.columns = [f'Head{head}' for head in range(1, 9)]
    
    return df_all_genes_attention

all_samples_genes_attention = pd.DataFrame()

for i in tqdm(range(0,1)):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE241184/'
    ip_file = file_path + 'Metastasis_Result/'  
    
    pred_label = list(np.load(ip_file+'pred.npy'))
    interested_labels = interest_pred_label
    
 
    barcodes = pd.read_csv(file_path+'cell_names.csv')

    features = pd.read_csv(file_path+'gene_names.csv')
  
    label = list(pd.read_csv(file_path+'label.csv')['Label'])
    label = ["Primary" if "P" in l else "Metastatic" if "M" in l else l for l in label]
    
    gene_names = list(features['Gene Names'])
    cell_names = list(barcodes['Cell Names'])   
    
    interest_label = interest_pred_label
    df_all_genes_attention = get_top_genes_attention_matrices(i, gene_names, cell_names, pred_label, label, interest_label)
    
   
    all_samples_genes_attention = pd.concat([all_samples_genes_attention, df_all_genes_attention])

  0%|          | 0/1 [00:00<?, ?it/s]


Number of cells of interest: 828



100%|██████████| 1/1 [00:04<00:00,  4.55s/it]


In [7]:
gene_frequencies = all_samples_genes_attention.index.value_counts()

all_samples_genes_attention['Frequency'] = all_samples_genes_attention.index.map(gene_frequencies)

print("\nConcatenated genes attention values and frequencies for all samples:")
print('\nTotal gene number: ', all_samples_genes_attention.shape[0])

all_samples_genes_attention_unique = pd.DataFrame()

all_samples_genes_attention['Attention'] = all_samples_genes_attention.iloc[:, :-2].max(axis=1)  # 这里 -2 是为了排除 "Frequency" 列

all_samples_genes_attention_sorted = all_samples_genes_attention.sort_values(by=['Frequency', 'Attention'], ascending=False)

all_samples_genes_attention_sorted_reset = all_samples_genes_attention_sorted.reset_index()

all_samples_genes_attention_unique = all_samples_genes_attention_sorted_reset.drop_duplicates(subset='index', keep='first')


all_samples_genes_attention_unique.set_index('index', inplace=True)


print("Unique sorted genes with max attention values and frequencies:")

mask = ~all_samples_genes_attention_unique.index.str.startswith("MT")


all_samples_genes_attention_unique = all_samples_genes_attention_unique[mask]

all_samples_genes_attention_unique.head(10)


Concatenated genes attention values and frequencies for all samples:

Total gene number:  153
Unique sorted genes with max attention values and frequencies:


,Head1,Head2,Head3,Head4,Head5,Head6,Head7,Head8,Frequency,Attention
index,,,,,,,,,,
TMSB4X,0.090708,0.067812,0.078776,0.060121,0.066706,0.153387,0.108212,0.094211,1,0.153387
FTL,0.090646,0.065284,0.082648,0.053249,0.061994,0.145474,0.098759,0.090211,1,0.145474
GPX1,0.085124,0.060787,0.087160,0.052342,0.066169,0.141465,0.107718,0.093674,1,0.141465
HSP90AA1,0.084256,0.064417,0.067545,0.053394,0.060216,0.138425,0.091092,0.075767,1,0.138425
TXNIP,0.072761,0.062524,0.078618,0.050150,0.061881,0.130085,0.097237,0.077165,1,0.130085
RPS29,NaN,0.048935,0.059699,0.045068,0.043830,0.127557,0.077837,0.075358,1,0.127557
HSP90B1,0.096415,0.087193,0.097994,0.070965,0.081657,0.125703,0.114580,0.086947,1,0.125703
TMSB10,0.082290,0.066111,0.066662,0.052755,0.058270,0.125657,0.097800,0.083252,1,0.125657
CYBA,0.077959,0.058850,0.069038,0.047743,0.059836,0.125076,0.114022,0.083372,1,0.125076


In [15]:
all_samples_genes_attention_unique.to_csv("Papi_M_N.csv")

# GSE162631 Brain cancer 

In [8]:
sample_to_pred_label = {
    "R1":[10,13,14,22],
    "R2":[14,15],
    "R3":[9,14,16,17],
    "R4":[15,12,3]
}


interest_pred_label  =   {
    "R1":[10,13,14,22],
    "R2":[14,15],
    "R3":[9,14,16,17],
    "R4":[15,12,3]
}

directories = ['R1','R2','R3','R4']
import pandas as pd
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix
import scanpy as sc
import os
from scipy.io import mmwrite
from tqdm import tqdm

#directories = ['Patient1','Patient2','Patient4','Patient5','Patient6','Patient7','Patient8']



from collections import Counter

def get_top_genes_attention_matrices(patient_id, gene_names, cell_names, pred_label, label, interest_label):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE162631/All_patient/{directories[patient_id]}/Metastasis_Result/Attention/'
    

    all_genes_attention = []
    
    interest_indices = [i for i, (pred, actual) in enumerate(zip(pred_label, label)) if pred in interest_label and actual == "Primary"]
    print(f'\nNumber of cells of interest: {len(interest_indices)}\n')
    
    for head in range(1, 9):
        attention_matrix_path = file_path + f'head-{head}.mtx'
        attention_matrix = mmread(attention_matrix_path)
        attention_matrix_csr = csr_matrix(attention_matrix)
        attention_matrix_filtered = attention_matrix_csr[:, interest_indices].transpose()
        df_attention_filtered = pd.DataFrame(attention_matrix_filtered.toarray(), index=np.array(cell_names)[interest_indices], columns=np.array(gene_names))
        
        top_100_genes_attention = df_attention_filtered.abs().mean(axis=0).nlargest(100)
        
        all_genes_attention.append(top_100_genes_attention)
    
    df_all_genes_attention = pd.concat(all_genes_attention, axis=1)
    df_all_genes_attention.columns = [f'Head{head}' for head in range(1, 9)]
    
    return df_all_genes_attention

all_samples_genes_attention = pd.DataFrame()

for i in tqdm(range(0,len(directories))):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE162631/All_patient/'+ str(directories[i]) + '/'
    ip_file = file_path + 'Metastasis_Result/'  
    
    pred_label = list(np.load(ip_file+'pred.npy'))
    interested_labels = interest_pred_label
    
    barcodes = pd.read_csv(file_path+'cell_names.csv')
    features = pd.read_csv(file_path+'gene_names.csv')
    label = list(pd.read_csv(file_path+'label.csv')['Label'])
    label = ["Primary" if "P" in l else "Metastatic" if "M" in l else l for l in label]
    
    gene_names = list(features['Gene Names'])
    cell_names = list(barcodes['Cell Names'])   
    
    interest_label = sample_to_pred_label[directories[i]]
    df_all_genes_attention = get_top_genes_attention_matrices(i, gene_names, cell_names, pred_label, label, interest_label)
    
    all_samples_genes_attention = pd.concat([all_samples_genes_attention, df_all_genes_attention])

  0%|          | 0/4 [00:00<?, ?it/s]


Number of cells of interest: 2021



 25%|██▌       | 1/4 [00:05<00:17,  5.76s/it]


Number of cells of interest: 646



 50%|█████     | 2/4 [00:10<00:10,  5.34s/it]


Number of cells of interest: 684



 75%|███████▌  | 3/4 [00:16<00:05,  5.31s/it]


Number of cells of interest: 4715



100%|██████████| 4/4 [00:24<00:00,  6.09s/it]


In [23]:

gene_frequencies = all_samples_genes_attention.index.value_counts()


all_samples_genes_attention['Frequency'] = all_samples_genes_attention.index.map(gene_frequencies)

print("\nConcatenated genes attention values and frequencies for all samples:")
print('\nTotal gene number: ', all_samples_genes_attention.shape[0])

all_samples_genes_attention_unique = pd.DataFrame()


all_samples_genes_attention['Attention'] = all_samples_genes_attention.iloc[:, :-2].max(axis=1)  # 这里 -2 是为了排除 "Frequency" 列

all_samples_genes_attention_sorted = all_samples_genes_attention.sort_values(by=['Frequency', 'Attention'], ascending=False)


all_samples_genes_attention_sorted_reset = all_samples_genes_attention_sorted.reset_index()

all_samples_genes_attention_unique = all_samples_genes_attention_sorted_reset.drop_duplicates(subset='index', keep='first')

all_samples_genes_attention_unique.set_index('index', inplace=True)

print("Unique sorted genes with max attention values and frequencies:")
mask = ~all_samples_genes_attention_unique.index.str.startswith("MT")

all_samples_genes_attention_unique = all_samples_genes_attention_unique[mask]

all_samples_genes_attention_unique.head(10)


Concatenated genes attention values and frequencies for all samples:

Total gene number:  588
Unique sorted genes with max attention values and frequencies:


,Head1,Head2,Head3,Head4,Head5,Head6,Head7,Head8,Frequency,Attention
index,,,,,,,,,,
SPP1,0.110931,0.103907,0.308490,0.449113,0.092592,0.152707,0.153017,0.104229,4,0.449113
FTL,0.111368,0.097977,0.355143,0.382926,0.112584,0.179099,0.143040,0.119439,4,0.382926
TMSB4X,0.089599,0.085599,0.254857,0.371482,0.083041,0.158265,0.127292,0.095819,4,0.371482
SRGN,0.097975,0.084800,0.317155,0.368056,0.089294,0.168596,0.123789,0.128242,4,0.368056
HLA-B,0.098659,0.083841,0.314797,0.365966,0.090208,0.169986,0.125381,0.123919,4,0.365966
HLA-DRA,0.077242,0.072054,0.230495,0.353287,0.080385,0.139932,0.115044,0.089569,4,0.353287
B2M,0.104822,0.096318,0.320371,0.349721,0.083719,0.170003,0.129718,0.118535,4,0.349721
ACTB,0.103111,0.092096,0.316144,0.342591,0.078924,0.162164,0.125857,0.107885,4,0.342591
MALAT1,0.089037,0.114163,0.271266,0.341697,0.090064,0.155890,0.131835,0.090139,4,0.341697


In [24]:
all_samples_genes_attention_unique.to_csv("Brain cancer_P_N.csv")

#### GSE167036（7 breast cancer）

#### GSE180286

In [17]:

import pandas as pd
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix
import scanpy as sc
import os
from scipy.io import mmwrite
from tqdm import tqdm
'''
directories = [
    "patientA2019_2",
    "patientA2019_3",
    "patientB2019_2",
    "patientB2019_3",
    "patientC2020_2",
    "patientC2020_3",
    "patientD2020_2",
    "patientD2020_3",
    "patientE2020_2",
    "patientE2020_3"
]


interest_pred_label = {
    "patientA2019_2":[7],
    "patientA2019_3":[8],
    "patientB2019_2":[13, 11, 3, 7, 15, 17],
    "patientB2019_3":[10, 3, 15, 7],
    "patientC2020_2":[4, 0, 11, 6, 1, 10],
    "patientC2020_3":[10, 2, 4, 14],
    "patientD2020_2":[2],
    "patientD2020_3":[4],
    "patientE2020_2":[4, 5, 2, 6, 11, 9, 3, 8],
    "patientE2020_3":[4, 2, 3, 12, 5, 7]
}
#early 
interest_pred_label = {
    "patientD2020_2":[2,11],
    "patientD2020_3":[4],

}
directories = [
    "patientD2020_2",
    "patientD2020_3",
]
'''
directories = [
    "patientA2019_2",
    "patientA2019_3",
    "patientB2019_2",
    "patientB2019_3",
    "patientC2020_2",
    "patientC2020_3",
    "patientD2020_2",
    "patientD2020_3",
    "patientE2020_2",
]
interest_pred_label = {
    "patientA2019_2":[8],
    "patientA2019_3":[7],
    "patientB2019_2":[7],
    "patientB2019_3":[10, 3, 15, 7],
    "patientC2020_2":[4],
    "patientC2020_3":[14],
    "patientD2020_2":[2],
    "patientD2020_3":[4],
    "patientE2020_2":[8]
}

from collections import Counter

def get_top_genes_attention_matrices(patient_id, gene_names, cell_names, pred_label, label, interest_label):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE180286/All_patient/{directories[patient_id]}/Metastasis_Result/Attention/'
    
    all_genes_attention = []
    
    interest_indices = [i for i, (pred, actual) in enumerate(zip(pred_label, label)) if pred in interest_label and actual == "Primary"]
    print(f'\nNumber of cells of interest: {len(interest_indices)}\n')
    
    for head in range(1, 9):
        attention_matrix_path = file_path + f'head-{head}.mtx'
        attention_matrix = mmread(attention_matrix_path)
        attention_matrix_csr = csr_matrix(attention_matrix)
        attention_matrix_filtered = attention_matrix_csr[:, interest_indices].transpose()
        df_attention_filtered = pd.DataFrame(attention_matrix_filtered.toarray(), index=np.array(cell_names)[interest_indices], columns=np.array(gene_names))

        top_50_genes_attention = df_attention_filtered.abs().mean(axis=0).nlargest(100)
        

        all_genes_attention.append(top_50_genes_attention)
    

    df_all_genes_attention = pd.concat(all_genes_attention, axis=1)

    df_all_genes_attention.columns = [f'Head{head}' for head in range(1, 9)]
    
    return df_all_genes_attention


all_samples_genes_attention = pd.DataFrame()


for i in tqdm(range(0,len(directories))):
    directory = directories[i]
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE180286/All_patient/' + str(directories[i]) + '/'
    ip_file = file_path + 'Metastasis_Result/'
    

    directory_path = file_path

    barcodes_file = next((f for f in os.listdir(directory_path) if f.endswith('cell_barcodes.csv')), None)
    features_file = next((f for f in os.listdir(directory_path) if f.endswith('gene_symbol.csv')), None)
    label_file = next((f for f in os.listdir(directory_path) if f.endswith('cell_label.csv')), None)

    barcodes = pd.read_csv(os.path.join(directory_path, barcodes_file))
    features = pd.read_csv(os.path.join(directory_path, features_file))
    label_data = pd.read_csv(os.path.join(directory_path, label_file))
    
    label = ["Primary" if "P" in l else "Metastatic" if "M" in l else l for l in list(label_data['0'])]
    pred_label = list(np.load(ip_file+'pred.npy'))
    

    gene_names = list(features['0'])
    cell_names = list(barcodes['0'])
    
    interest_label = interest_pred_label[directory]
    df_all_genes_attention = get_top_genes_attention_matrices(i, gene_names, cell_names, pred_label, label, interest_label)
    

    all_samples_genes_attention = pd.concat([all_samples_genes_attention, df_all_genes_attention])


gene_frequencies = all_samples_genes_attention.index.value_counts()


all_samples_genes_attention['Frequency'] = all_samples_genes_attention.index.map(gene_frequencies)

print("\nConcatenated genes attention values and frequencies for all samples:")
print('\nTotal gene number: ', all_samples_genes_attention.shape[0])

all_samples_genes_attention_unique = pd.DataFrame()


all_samples_genes_attention['Attention'] = all_samples_genes_attention.iloc[:, :-2].max(axis=1)  # 这里 -2 是为了排除 "Frequency" 列


all_samples_genes_attention_sorted = all_samples_genes_attention.sort_values(by=['Frequency', 'Attention'], ascending=False)


all_samples_genes_attention_sorted_reset = all_samples_genes_attention_sorted.reset_index()


all_samples_genes_attention_unique = all_samples_genes_attention_sorted_reset.drop_duplicates(subset='index', keep='first')


all_samples_genes_attention_unique.set_index('index', inplace=True)


print("Unique sorted genes with max attention values and frequencies:")

mask = ~all_samples_genes_attention_unique.index.str.startswith("MT")


all_samples_genes_attention_unique = all_samples_genes_attention_unique[mask]

all_samples_genes_attention_unique.head(10)

  0%|          | 0/9 [00:00<?, ?it/s]


Number of cells of interest: 177



 11%|█         | 1/9 [00:01<00:08,  1.06s/it]


Number of cells of interest: 444



 22%|██▏       | 2/9 [00:02<00:08,  1.22s/it]

...

 89%|████████▉ | 8/9 [00:19<00:02,  2.40s/it]


Number of cells of interest: 269



100%|██████████| 9/9 [00:21<00:00,  2.39s/it]


Concatenated genes attention values and frequencies for all samples:

Total gene number:  1320
Unique sorted genes with max attention values and frequencies:


,Head1,Head2,Head3,Head4,Head5,Head6,Head7,Head8,Frequency,Attention
index,,,,,,,,,,
TMSB4X,0.128473,0.177889,0.134688,0.222094,0.470035,0.196772,0.124852,0.100984,9,0.470035
B2M,0.468505,0.328250,0.150729,0.140119,0.171684,0.235357,0.116199,0.152746,9,0.468505
MALAT1,0.117991,0.161483,0.123353,0.219908,0.437318,0.171185,0.133649,0.107828,9,0.437318
PTMA,0.388090,0.256208,0.122836,0.150836,0.193232,0.306325,0.124737,0.156426,9,0.388090
RPS29,0.128232,0.139385,0.147666,0.236840,0.381925,0.176025,0.190604,0.143162,9,0.381925
HSP90AA1,0.376596,0.244290,0.147275,0.166095,0.151576,0.196377,0.127332,0.121002,9,0.376596
RPS16,0.103108,0.142802,0.132707,0.219597,0.347653,0.167095,0.135094,0.114779,9,0.347653
HSP90AB1,0.112814,0.144297,0.144008,0.122699,0.344092,0.104822,0.109929,0.083449,9,0.344092
RPS27,0.103865,0.152847,0.123386,0.188099,0.341267,0.156372,0.136946,0.135038,9,0.341267


In [56]:
all_samples_genes_attention_unique.to_csv("Breast_cancer9 P.csv")

#### Head and Neck Cancer

In [20]:
'''

sample_to_pred_label = {
    'HN242': [7],
    'HN257': [16],
    'HN272': [0,9],
    'HN279': [0,2]
}

cell_type_annotations = {
    "HN242":{1: 'CD8 T Cells', 7: 'Epithelial Cells', 10: 'B Cells', 6: 'Cancer-associated Fibroblasts (CAFs)', 4: 'Cancer-associated Fibroblasts (CAFs)', 9: 'Epithelial Cells', 8: 'TAM Cells', 0: 'B Cells', 2: 'Cytotoxic T Cells', 5: 'CD8 T Cells', 3: 'Treg Cells', 11: 'Cytotoxic T Cells', 12: 'B Cells'},
    "HN257":{5: 'TAM Cells', 6: 'TAM Cells', 8: 'Cancer-associated Fibroblasts (CAFs)', 1: 'Cytotoxic T Cells', 0: 'Cancer-associated Fibroblasts (CAFs)', 3: 'Epithelial Cells', 15: 'TAM Cells', 13: 'B Cells', 9: 'Cancer-associated Fibroblasts (CAFs)', 11: 'Treg Cells', 14: 'Cancer-associated Fibroblasts (CAFs)', 16: 'Epithelial Cells', 10: 'B Cells', 4: 'Treg Cells', 17: 'Cancer-associated Fibroblasts (CAFs)', 18: 'Cancer-associated Fibroblasts (CAFs)', 2: 'Cytotoxic T Cells', 7: 'B Cells', 12: 'Cancer-associated Fibroblasts (CAFs)', 19: 'CD4 T Cells', 20: 'TAM Cells'},
    "HN272":{0: 'Epithelial Cells', 6: 'Cancer-associated Fibroblasts (CAFs)', 7: 'Cancer-associated Fibroblasts (CAFs)', 2: 'Cancer-associated Fibroblasts (CAFs)', 14: 'Epithelial Cells', 4: 'TAM Cells', 1: 'Cytotoxic T Cells', 12: 'Cancer-associated Fibroblasts (CAFs)', 11: 'B Cells', 10: 'B Cells', 8: 'Epithelial Cells', 5: 'Cancer-associated Fibroblasts (CAFs)', 16: 'Cytotoxic T Cells', 13: 'Treg Cells', 15: 'Cancer-associated Fibroblasts (CAFs)', 17: 'Cancer-associated Fibroblasts (CAFs)', 9: 'CD8 T Cells', 3: 'Cancer-associated Fibroblasts (CAFs)'},
    "HN279":{6: 'TAM Cells', 5: 'Cancer-associated Fibroblasts (CAFs)', 12: 'Cytotoxic T Cells', 1: 'Epithelial Cells', 11: 'Cancer-associated Fibroblasts (CAFs)', 2: 'Epithelial Cells', 13: 'Cancer-associated Fibroblasts (CAFs)', 15: 'TAM Cells', 4: 'Cancer-associated Fibroblasts (CAFs)', 17: 'B Cells', 0: 'Epithelial Cells', 10: 'Cancer-associated Fibroblasts (CAFs)', 7: 'Treg Cells', 3: 'CD8 T Cells', 16: 'Cancer-associated Fibroblasts (CAFs)', 14: 'Cancer-associated Fibroblasts (CAFs)', 8: 'Cytotoxic T Cells', 18: 'CD4 T Cells', 9: 'B Cells'}
}


interest_pred_label = {
    'HN242': [7],
    'HN257': [16],
    'HN272': [0,8],
    'HN279': [0,2]
}
'''
directories = [
    "HN242",
    "HN251",
    "HN257",
    "HN272",
    "HN279"
]
sample_to_pred_label = {
    'HN242': [7],
    "HN251": [11],
    #'HN257': [16],
    'HN272': [0],
    'HN279': [0]
} 
interest_pred_label = {
    'HN242': [7],
    "HN251": [11],
    #'HN257': [16],
    'HN272': [0],
    'HN279': [0]
}
directories = [
    "HN242",
    "HN251",
   # "HN257",
    "HN272",
    "HN279"
]

from collections import Counter

def get_top_genes_attention_matrices(patient_id, gene_names, cell_names, pred_label, label, interest_label):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE188737/{directories[patient_id]}/Metastasis_Result/Attention/'
    

    all_genes_attention = []
    
    interest_indices = [i for i, (pred, actual) in enumerate(zip(pred_label, label)) if pred in interest_label and actual == "Metastatic"]
    print(f'\nNumber of cells of interest: {len(interest_indices)}\n')
    
    for head in range(1, 9):
        attention_matrix_path = file_path + f'head-{head}.mtx'
        attention_matrix = mmread(attention_matrix_path)
        attention_matrix_csr = csr_matrix(attention_matrix)
        attention_matrix_filtered = attention_matrix_csr[:, interest_indices].transpose()
        df_attention_filtered = pd.DataFrame(attention_matrix_filtered.toarray(), index=np.array(cell_names)[interest_indices], columns=np.array(gene_names))
        
        top_100_genes_attention = df_attention_filtered.abs().mean(axis=0).nlargest(100)
        
        all_genes_attention.append(top_100_genes_attention)
    
    df_all_genes_attention = pd.concat(all_genes_attention, axis=1)
    df_all_genes_attention.columns = [f'Head{head}' for head in range(1, 9)]
    
    return df_all_genes_attention

all_samples_genes_attention = pd.DataFrame()

for i in tqdm(range(0,len(directories))):
    file_path = f'/fs/ess/PAS1475/Maoteng/Metastasis_new/GSE188737/'+ str(directories[i]) + '/'
    ip_file = file_path + 'Metastasis_Result/'  
    
    pred_label = list(np.load(ip_file+'pred.npy'))

    interested_labels = sample_to_pred_label[directories[i]]
    
    directory_path = file_path
    pred_label = list(np.load(ip_file+'pred.npy'))

    barcodes = pd.read_csv(file_path+'cell_barcodes.csv')

    features = pd.read_csv(file_path+'gene_symbol.csv')

    label = list(pd.read_csv(file_path+'cell_label.csv')['P_Mid'])
    label = ["Primary" if "P" in l else "Metastatic" if "M" in l else l for l in label]
    gene_names = list(features['x'])
    cell_names = list(barcodes['x'])  
    
    interest_label = sample_to_pred_label[directories[i]]
    df_all_genes_attention = get_top_genes_attention_matrices(i, gene_names, cell_names, pred_label, label, interest_label)
    

    all_samples_genes_attention = pd.concat([all_samples_genes_attention, df_all_genes_attention])

  0%|          | 0/4 [00:00<?, ?it/s]


Number of cells of interest: 89



 25%|██▌       | 1/4 [00:00<00:02,  1.11it/s]


Number of cells of interest: 17



 50%|█████     | 2/4 [00:02<00:02,  1.44s/it]


Number of cells of interest: 58



 75%|███████▌  | 3/4 [00:04<00:01,  1.56s/it]


Number of cells of interest: 16



100%|██████████| 4/4 [00:05<00:00,  1.43s/it]


In [22]:

all_samples_genes_attention['Abs Interest Mean Attention Value'] = all_samples_genes_attention['Interest Mean Attention Value'].abs()

sorted_df = all_samples_genes_attention.sort_values(by=['Sample', 'Gene', 'Abs Interest Mean Attention Value'], ascending=[True, True, False])

result_alternative = sorted_df.drop_duplicates(subset=['Sample', 'Gene'], keep='first')


result_filtered = result_alternative[~result_alternative['Gene'].str.startswith('MT')]

result_filtered['Frequency'] = result_filtered.groupby('Gene')['Gene'].transform('count')


result_filtered_sorted = result_filtered.sort_values(by=['Gene', 'Abs Interest Mean Attention Value'], ascending=[True, False])

result_filtered_max_alternative = result_filtered_sorted.drop_duplicates(subset='Gene', keep='first')

final_result = result_filtered_max_alternative.sort_values(by=['Frequency', 'Abs Interest Mean Attention Value'], ascending=[False, False])

final_result_filtered = final_result.loc[:,['Gene','Frequency','P-Value','Abs Interest Mean Attention Value','Interest Mean Attention Value','Non Interest Mean Attention Value']]

final_result_filtered["logFC"] = np.log2(abs(final_result_filtered["Interest Mean Attention Value"] / final_result_filtered["Non Interest Mean Attention Value"]))
final_result_filtered

,Gene,Frequency,P-Value,Abs Interest Mean Attention Value,Interest Mean Attention Value,Non Interest Mean Attention Value,logFC
270,C1QBP,4,1.792046e-05,0.172223,0.172223,0.029314,2.554606
242,C14orf2,3,2.165171e-02,0.332567,0.332567,0.170400,0.964718
70,ATP1B3,3,6.143716e-10,0.270081,0.270081,0.025285,3.417037
112,WTAP,3,4.142981e-02,0.247380,0.247380,0.125088,0.983785
301,PSMA7,3,9.366531e-03,0.240248,0.240248,0.060510,1.989288
...,...,...,...,...,...,...,...
305,PRKCE,1,4.085551e-03,0.002672,0.002672,0.001519,0.815282
1439,NRP1,1,4.219931e-02,0.002520,0.002520,0.002431,0.052201
1395,PAFAH1B2,1,7.430406e-08,0.002454,0.002454,0.000651,1.913360
1456,NRBF2,1,6.442290e-04,0.002286,0.002286,0.000612,1.900888
